In [1]:
base = r'KolejkiEkstraklasa'

In [11]:
import os
import pandas as pd
matchdays = []
for filename in os.listdir(base):
    if int(filename)<15:
        matchday = pd.read_csv(f"{base}/{filename}", header=None)
        matchdays.append(matchday)
league = pd.concat(matchdays, ignore_index=True)

,0,1,2,3,4
0,Cracovia-Pogoń,2:1,7.34,6,1:0
1,Górnik-Podbeskidzie,4:2,3.25,11,2:1
2,Jagiellonia-Wisła,1:1,11.08,2,1:0
3,Raków-Legia,1:2,9.03,3,1:1
4,Śląsk-Piast,2:0,6.94,5,1:1
...,...,...,...,...,...
107,Legia-Lech,2:1,5.02,5,2:2
108,Podbeskidzie-Zagłębie,2:1,7.60,6,1:1
109,Raków-Wisła,0:0,7.06,6,1:1
110,Stal-Warta,0:1,7.62,6,0:2


In [12]:
g = open("Statystyki/poissonstats-ekstraklasa", "r", encoding='utf-8')
clubs = [x.strip().split(",")[0] for x in g]
clubs

['Legia',
 'Lech',
 'Piast',
 'Lechia',
 'Śląsk',
 'Pogoń',
 'Cracovia',
 'Jagiellonia',
 'Górnik',
 'Raków',
 'Zagłębie',
 'Płock',
 'Wisła',
 'Stal',
 'Podbeskidzie',
 'Warta']

In [77]:
columns = ["avg_prob", "avg_pos", "diff_points", "diff_goals", "diff_conceded", "clear_pred_amount", "clear_pred_prob", "points"]
clubs_pred = {}

for c in clubs:
    clear_pred = 0
    clear_real = 0
    clubs_pred[c] = [0,0,0,0,0,0,0,0]
    points = 0
    for index, row in league.iterrows():
        home = row[0].split('-')[0]
        away = row[0].split('-')[1]
        if c==home or c==away:
            clubs_pred[c][0] += float(row[2])/14
            clubs_pred[c][1] += int(row[3])/14
            if c==row[0].split('-')[0]:
                a = (int(row[1].split(':')[0])-int(row[4].split(':')[0]))
                b = (int(row[1].split(':')[1])-int(row[4].split(':')[1]))
                clubs_pred[c][3] += a
                clubs_pred[c][4] += b
                if int(row[4].split(':')[1]) is 0:
                    clear_pred += 1
                    if int(row[1].split(':')[1]) is 0:
                        clear_real +=1
                if int(row[4].split(':')[0])>int(row[4].split(':')[1]):
                    points+=3
                elif int(row[4].split(':')[0]) is int(row[4].split(':')[1]):
                    points+=1
            elif c==row[0].strip().split('-')[1]:
                a = (int(row[1].split(':')[1])-int(row[4].split(':')[1]))
                b = (int(row[1].split(':')[0])-int(row[4].split(':')[0]))
                clubs_pred[c][3] += a
                clubs_pred[c][4] += b
                if int(row[4].split(':')[0]) is 0:
                    clear_pred += 1
                    if int(row[1].split(':')[0]) is 0:
                        clear_real +=1
                if int(row[4].split(':')[1])>int(row[4].split(':')[0]):
                    points+=3
                elif int(row[4].split(':')[1]) is int(row[4].split(':')[0]):
                    points+=1   
    clubs_pred[c][5] = clear_pred
    clubs_pred[c][6] = clear_real/max(clear_pred,1)
    clubs_pred[c][7] = points
    
league_pred_df = pd.DataFrame.from_dict(clubs_pred, orient='index', columns = columns)

In [78]:
league_pred_df

,avg_prob,avg_pos,diff_points,diff_goals,diff_conceded,clear_pred_amount,clear_pred_prob,points
Legia,7.687143,7.142857,0,9,7,7,0.571429,26
Lech,4.811429,9.642857,0,-5,7,3,0.333333,27
Piast,5.720714,9.428571,0,9,2,4,0.000000,13
Lechia,2.977143,15.071429,0,-1,6,2,0.000000,26
Śląsk,6.830714,6.857143,0,9,4,5,0.000000,20
Pogoń,10.413571,6.000000,0,12,5,12,0.666667,22
Cracovia,9.326429,5.000000,0,3,5,5,0.400000,25
Jagiellonia,4.768571,11.000000,0,12,12,4,0.000000,16
Górnik,8.374286,8.642857,0,5,5,6,0.333333,25
Raków,5.603571,9.785714,0,8,0,3,0.333333,23


In [82]:
league_pred_df.sort_values("points",ascending=False)

,avg_prob,avg_pos,diff_points,diff_goals,diff_conceded,clear_pred_amount,clear_pred_prob,points
Lech,4.811429,9.642857,0,-5,7,3,0.333333,27
Legia,7.687143,7.142857,0,9,7,7,0.571429,26
Lechia,2.977143,15.071429,0,-1,6,2,0.000000,26
Cracovia,9.326429,5.000000,0,3,5,5,0.400000,25
Górnik,8.374286,8.642857,0,5,5,6,0.333333,25
Zagłębie,8.057857,4.928571,0,5,6,5,0.000000,24
Raków,5.603571,9.785714,0,8,0,3,0.333333,23
Pogoń,10.413571,6.000000,0,12,5,12,0.666667,22
Śląsk,6.830714,6.857143,0,9,4,5,0.000000,20
Jagiellonia,4.768571,11.000000,0,12,12,4,0.000000,16
